In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import astropy.units as au
import astropy.constants as ca

In [2]:
#Semi empirical model + error for spectrum TRAPPIST-1 from Wilson et al 2021
trappist_SED = np.genfromtxt("../data/TRAPPIST1_Semimodel_Wilson/TRAPPIST-1_model_const_res_v07.ecsv", \
                              delimiter = "")
#unit erg A^-1 s^-1 cm^-2 

In [3]:
flux_TRAP = trappist_SED[:,1]  * au.erg * au.cm**(-2) * au.s**(-1) * au.Angstrom**(-1)
wavelength_TRAP = trappist_SED[:,0]  * au.Angstrom
#error of semiemprical model is 5% Wilson et al 2021
fluxerror_TRAP = (0.05*trappist_SED[:,1])  * au.erg * au.cm**(-2) * au.s**(-1) * au.Angstrom**(-1)

In [4]:
#Muscat Reaponse
muscat = pd.read_csv("../data/MuSCAT2_response/MuSCAT2_response.csv")

muscat = muscat.fillna(0) 
muscat["lamb"] = muscat.wavelength*1e1   #Angstrom

In [5]:
# Gaia response function for G filter
gaia = np.genfromtxt("../data/Gaia_response/GaiaDR2_Passbands.dat")
mask = gaia == 99.99 
gaia[mask] = 0
gaia[:,0] = gaia[:,0]*1e1  #Angstrom

In [6]:
#Interplotation to TRAPPIST-1 Gaia
rres = np.interp(gaia[:,0], wavelength_TRAP.value, flux_TRAP.value) 

In [7]:
#total flux in gaia passbands
gaiaflux  = np.trapz(gaia[:,1]*rres, x=gaia[:,0])
TRAPPIST_gaia = 15.622554 #magnitude gaia VEGAMAG
ZP_gaia = 25.6883657251 #z point gaia VEGAMAG

In [59]:
#f2  = flux_TRAP/gaiaflux

In [8]:
#retrieve flux value of TRAPPIST-1 in gaia absolute units
def gaia_flux(m,ZP):
    return 10**(-(m-ZP)/2.5) * au.erg * au.m**(-2) *au.Angstrom**(-1) * au.s**(-1)

In [9]:
gaia_TRAP =  gaia_flux(TRAPPIST_gaia,ZP_gaia) 

In [10]:
norm = gaiaflux/gaia_TRAP.to("erg s^-1 cm^-2 Angstrom^-1") 

In [11]:
#normalisation and callibration of MM SED to gaia 
flux_norm = flux_TRAP/norm 

In [12]:
#Interpolation to TRAPPIST-1 M2
rres1 = np.interp(muscat.lamb, wavelength_TRAP.value, flux_norm.value) 

In [13]:
#muscat specific-flux
muscatflux  = np.asarray([np.trapz(muscat.Total_g*rres1, x=muscat.lamb), \
                   np.trapz(muscat.Total_r*rres1, x=muscat.lamb), \
                   np.trapz(muscat.Total_i*rres1, x=muscat.lamb), \
                   np.trapz(muscat.Total_z*rres1, x=muscat.lamb) ])

In [14]:
#Wilson et al 2021 take 5% of the flux values as error
muscatfluxerror  = np.asarray([np.trapz(muscat.Total_g*0.05*rres1, x=muscat.lamb), \
                   np.trapz(muscat.Total_r*0.05*rres1, x=muscat.lamb), \
                   np.trapz(muscat.Total_i*0.05*rres1, x=muscat.lamb), \
                   np.trapz(muscat.Total_z*0.05*rres1, x=muscat.lamb) ])

In [15]:
#Lienhardt et al 2020
dist_TRAP = 3.835084e+19 *au.cm 
dist_TRAP_error = 0.02 * 3.086e+18 *au.cm

lumi_TRAP = muscatflux*au.erg*au.cm**(-2)*au.s**(-1)*(4*np.pi*(dist_TRAP)**2)
#lumierror_TRAP = np.sqrt(4*np.pi*(dist_TRAP.value)**2 * brightnesserror_TRAP.value**2 + 8*np.pi*brightness_TRAP.value*dist_TRAP.value*dist_TRAP_error.value**2 ) *au.erg * au.s**(-1)
lumi_TRAP.to("W")


<Quantity [4.85355719e+31, 1.06939275e+32, 8.15255369e+32, 1.17453990e+33] W>

In [16]:
#safe passbandflux
with open("../results/values/brightness_TRAPPIST-1_withCCDeffeciency.txt", "w+") as f:
    f.write("g , r , i, z, units: [erg cm-2 s-1]  \n")
    f.write(str(muscatflux[0])+ ',' +  str(muscatflux[1])+ ',' + \
            str(muscatflux[2])+ ',' + str(muscatflux[3]) + '\n' )
    f.write(str(muscatfluxerror[0])+ ',' +  str(muscatfluxerror[1])+ ',' + \
            str(muscatfluxerror[2])+ ',' + str(muscatfluxerror[2]) + '\n' )

In [17]:
#safe lumi
with open("../results/values/lumi_TRAPPIST-1_withCCDeffeciency.txt", "w+") as f:
    f.write("g , r , i, z, units: [erg s-1] \n")
    f.write(str(lumi_TRAP[0].value)+ ',' +  str(lumi_TRAP[1].value)+ ',' + \
            str(lumi_TRAP[2].value)+ ',' + str(lumi_TRAP[3].value) )